**Mounting Drive**

**Installing required Libraries**

In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import os
import numpy as np
from model.pointpillars import PointPillars
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def save_image_features(image_folder, my_model, output_folder, device):

    os.makedirs(output_folder, exist_ok=True)

    model = models.resnet18(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, 10)
    model.load_state_dict(torch.load(my_model))
    model.fc = torch.nn.Identity()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert 1-channel to 3-channel
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert to Tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the image
    ])

    filenames = sorted(os.listdir(image_folder))  # Sort files for consistent ordering
    for filename in filenames:
        image_path = os.path.join(image_folder, filename)
        image = Image.open(image_path).convert('RGB')
        image_tensor = transform(image).unsqueeze(0).to(device)  # Add batch dimension
        with torch.no_grad():
            feature = model(image_tensor).cpu()  # Extract features

        H = 62          #these values are obtaiend from pointpillar inference, in order to match
        W = 54
        feature = feature.unsqueeze(1).repeat(1, H * W, 1)  # [1, 3348, 512]

        feature_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.npy")
        np.save(feature_path, feature)
    print(f"Image features saved in {output_folder}")

In [3]:
# Function to load and preprocess PCD file
def load_pcd_file(pcd_path):
    # Load the point cloud data using Open3D
    pcd = o3d.io.read_point_cloud(pcd_path)
    # Convert points to numpy array
    points = np.asarray(pcd.points)  # (N, 3)
    # Check if colors are available and add them
    if pcd.has_colors():
        colors = np.asarray(pcd.colors)  # (N, 3)
        data = np.hstack((points, colors))  # Combine points and colors
    else:
        data = points  # Only points available

    return data


# Preprocess the point cloud data for the model
def preprocess_point_cloud(pcd_points):
    # Assuming the model requires specific preprocessing like voxelization
    # Example: Downsample using voxel grid (this is one of many possible approaches)

    # For example, voxel size (you may need to tune this)
    voxel_size = 0.2
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcd_points)
    downsampled_pcd = pcd.voxel_down_sample(voxel_size)

    # Convert the downsampled points to numpy array
    downsampled_points = np.asarray(downsampled_pcd.points)  # Shape: (N', 3)

    # Convert to torch tensor and add batch dimension
    input_tensor = torch.tensor(downsampled_points, dtype=torch.float32).unsqueeze(0)
    return input_tensor

# Inference function
def pp_inference(input_tensor, model):
    with torch.no_grad():
        # Run the model inference (assuming the model returns features directly)
        features = model(input_tensor)
        features = features[-1]      #model provides features on three different scales i.e. at different levels of backbone, last lavel features are more enriched that is why taken.
    return features

In [4]:
def save_pcd_features(pcd_folder, pp_model, output_folder, device):
    os.makedirs(output_folder, exist_ok=True)
    # Initialize the PointPillars model
    model = PointPillars(
        nclasses=3,
        voxel_size=[0.16, 0.16, 4],
        point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
        max_num_points=32,
        max_voxels=(16000, 40000)  # Training and inference max_voxels
    )

    model_state_dict = torch.load(pp_model, map_location=torch.device('cpu'))
    # Load the filtered state dict into the model (without the head weights)
    model.load_state_dict(model_state_dict, strict=False)

    # # Now replace the head layer with Identity

    model.neck = nn.Identity()
    model.head = nn.Identity()

    # Set the model to evaluation mode
    model.eval()

    features = []
    for file_name in os.listdir(pcd_folder):
        full_path = os.path.join(pcd_folder, file_name)
        points = load_pcd_file(full_path)
        
        
        # Preprocess the point cloud data
        input_tensor = preprocess_point_cloud(points)
        print(f"input tensor shape = {input_tensor.shape}")

        # Run inference
        features = pp_inference(input_tensor, model)

        B, C, H, W = features.shape  # B=1, C=256, H=62, W=54

        # Reshape PointPillars features to [Batch, Seq_Len, Channels]
        feature = features.permute(0, 2, 3, 1).reshape(B, H * W, C)  # [1, 3348, 256]
        print(feature)
        feature_path = os.path.join(output_folder, f"{os.path.splitext(file_name)[0]}.npy")
        print(feature_path)
        np.save(feature_path, feature)
    print(f"PCD features saved in {output_folder}")


In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def save_obd_features(obd_file, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    obd_data = pd.read_csv(obd_file)
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(obd_data)
    feature_path = os.path.join(output_folder, "obd_features.npy")
    np.save(feature_path, scaled_data)
    print(f"OBD features saved in {output_folder}")



In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Paths to your data
image_folder = r"C:\Users\hussa\OneDrive\Desktop\Projects\Project\MOT\INFRA-3DRC_scene-15\INFRA-3DRC_scene-15\camera_01\camera_01__data"
pcd_folder = r"C:\Users\hussa\OneDrive\Desktop\Projects\Project\MOT\INFRA-3DRC_scene-15\INFRA-3DRC_scene-15\lidar_01\lidar_01__data"
# obd_file = "path/to/obd_data.csv"

# Paths to save features
image_output_folder = r"C:\Users\hussa\OneDrive\Desktop\dataset/camera"
pcd_output_folder = r"C:\Users\hussa\OneDrive\Desktop\dataset/lidar"
# obd_output_folder = "features/obd"

# Load pretrained models
resnet18_model = r"C:\Users\hussa\OneDrive\Desktop\Projects\ROS2-Modular-Framework-for-End-to-End-Autonomous-Vehicle-Control-from-Raw-Sensor-Data\resnet18\trained models\feature_extractor_resnet18.pth"
pointpillars_model = r"C:\Users\hussa\OneDrive\Desktop\Projects\ROS2-Modular-Framework-for-End-to-End-Autonomous-Vehicle-Control-from-Raw-Sensor-Data\poinpillar\pre_trained\epoch_160.pth"

# Extract and save features
save_image_features(image_folder, resnet18_model, image_output_folder, device)
save_pcd_features(pcd_folder, pointpillars_model, pcd_output_folder, device)
# save_obd_features(obd_file, obd_output_folder)



c:\Users\hussa\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\hussa\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Image features saved in C:\Users\hussa\OneDrive\Desktop\dataset/camera
input tensor shape = torch.Size([1, 25950, 3])
tensor([[[0.1184, 0.0525, 0.0277,  ..., 0.0801, 0.0000, 0.0000],
         [0.3384, 0.0878, 0.0000,  ..., 0.0000, 0.1475, 0.0000],
         [0.2301, 0.1445, 0.1551,  ..., 0.0000, 0.2247, 0.0000],
         ...,
         [0.7802, 0.1593, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.3990, 0.1137, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.5199, 0.0975, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]])
C:\Users\hussa\OneDrive\Desktop\dataset/lidar\lidar_01__2023-06-02-21-28-09-321.npy
input tensor shape = torch.Size([1, 25873, 3])
tensor([[[0.1184, 0.0525, 0.0277,  ..., 0.0801, 0.0000, 0.0000],
         [0.3384, 0.0878, 0.0000,  ..., 0.0000, 0.1475, 0.0000],
         [0.2301, 0.1445, 0.1551,  ..., 0.0000, 0.2247, 0.0000],
         ...,
         [0.1233, 0.1182, 0.2594,  ..., 0.0000, 0.0000, 0.0000],
         [0.1127, 0.1336, 0.2213,  ..., 0.0000, 0.0000, 0.0852],
      

In [7]:
# Path to your .npy file
file_path = r"C:\Users\hussa\OneDrive\Desktop\dataset\camera_01__2023-06-02-21-28-09-314.npy"

# Load the file
data = np.load(file_path)

# Display basic information
print(f"Shape: {data.shape}")
print(f"Data type: {data.dtype}")
print(data)  # Print the contents


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\hussa\\OneDrive\\Desktop\\dataset\\camera_01__2023-06-02-21-28-09-314.npy'

In [ ]:
folder_path = r"C:\Users\hussa\OneDrive\Desktop\Projects\Project\MOT\INFRA-3DRC_scene-15\INFRA-3DRC_scene-15\lidar_01\lidar_01__data"

for file_name in os.listdir(folder_path):
    full_path = os.path.join(folder_path, file_name)
    if os.path.isfile(full_path) and file_name.endswith(".npy"):  # Check for .npy extension
        os.remove(full_path)  # Delete the file
        print(f"Deleted: {full_path}")

Deleted: C:\Users\hussa\OneDrive\Desktop\Projects\Project\MOT\INFRA-3DRC_scene-15\INFRA-3DRC_scene-15\lidar_01\lidar_01__data\lidar_01__2023-06-02-21-28-09-321.npy
Deleted: C:\Users\hussa\OneDrive\Desktop\Projects\Project\MOT\INFRA-3DRC_scene-15\INFRA-3DRC_scene-15\lidar_01\lidar_01__data\lidar_01__2023-06-02-21-28-09-368.npy
Deleted: C:\Users\hussa\OneDrive\Desktop\Projects\Project\MOT\INFRA-3DRC_scene-15\INFRA-3DRC_scene-15\lidar_01\lidar_01__data\lidar_01__2023-06-02-21-28-09-468.npy
Deleted: C:\Users\hussa\OneDrive\Desktop\Projects\Project\MOT\INFRA-3DRC_scene-15\INFRA-3DRC_scene-15\lidar_01\lidar_01__data\lidar_01__2023-06-02-21-28-09-567.npy
Deleted: C:\Users\hussa\OneDrive\Desktop\Projects\Project\MOT\INFRA-3DRC_scene-15\INFRA-3DRC_scene-15\lidar_01\lidar_01__data\lidar_01__2023-06-02-21-28-09-620.npy
Deleted: C:\Users\hussa\OneDrive\Desktop\Projects\Project\MOT\INFRA-3DRC_scene-15\INFRA-3DRC_scene-15\lidar_01\lidar_01__data\lidar_01__2023-06-02-21-28-09-670.npy
Deleted: C:\User